# 품질검사성적서 등록 목록

In [47]:
from common import commonFunc as cf
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
### APIKEY 불러오기 ###
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [48]:
APIKEYLEN = len(APIKEY)

In [49]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [50]:
JSONKEY = "items"
DUMMY = 0

In [51]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

### 2. 입력 파라미터 정보 불러오기

In [53]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [54]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen
#mydf = dfCombi.cstrnm =="(주)디제이 종합건설"
#dfCombi[mydf]

74882

In [55]:
def saveLastSearchDate(paramStartDate):
    cf.saveparam(paramStartDate, SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHDATE")
    print("Last Search Date {} is saved.".format(paramStartDate))
def loadLastSearchDate():
    ret = cf.loadparam(SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHDATE")
    print(ret+" is loaded")
    return ret
def saveLastSearchDateByName(resultDf):
    cf.savedata(resultDf, SITENAME,DATANAME,SERVICENAME+"_LastSearchDateByName")
    print("Last Search Date {} is saved.")

In [57]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endMonth = date.strftime("%m")
endDay = date.strftime("%d")
# 2001년 이전 데이터는 없음, 초기 데이터 수신 후 이후부터는 
# 시작점(startDate): 이전 검색일
# 종료점(endDate): 검색 당년.당월.당일 로 셋팅해야 함
startYr = 2001 
print("{} {} {}".format(endYr,endMonth,endDay))

startDate = 20010101
try:
    startDate = loadLastSearchDate()
except Exception as e:
    print(e)
    
endDate = endYr+endMonth+endDay
# 아래 검색데이터 저장 완료 후
# 현재 종료점(endDate)를 saveLastSearchDate를 호출하여 저장해야 함

2022 04 28
20220427 is loaded


### [추가] 3. 파라미터정보 불러오기

## 품질검사성적서 등록정보 전용 스크랩 함수

In [58]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1.2)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [59]:
resultDfMerged = pd.DataFrame()
g_totalCount = 0



In [60]:
import hashlib

# 파라미터 정보가 기존에 요청했던 정보에 포함되었는지 검사하는 함수
# 리턴값 -1 : 파일이 없을 경우
#       0 : 파일은 정상존재하나, 요청한적 없는 경우
#       1 : 이미 요청했던 경우
# 시공자명, 검색시작일, 검색종료일
def checkExistenceData(cstrnm, start, end, pageNo):
    print("{},{},{},{}".format(cstrnm, start, end, pageNo))
    m = hashlib.sha256()
    m.update((cstrnm+start+end+str(pageNo)).encode('utf-8'))
    print(m.hexdigest())
    try:
        orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)
    except Exception as e:
        print(e)
        return -1
    if m.hexdigest() in orgParam:
        print("이미 존재함")
        return 1
    return 0

# 요청 파라미터를 저장하는 함수
# 시공자명, 검색시작일, 검색종료일
def hashSaveParam(cstrnm, start, end, pageNo):
    print("{},{},{},{}".format(cstrnm, start, end, pageNo))
    m = hashlib.sha256()
    m.update((cstrnm+start+end+str(pageNo)).encode('utf-8'))
    print(m.hexdigest())
    orgParam = []
    try:
        orgParam = cf.loadparam(SITENAME,DATANAME,SERVICENAME)
    except Exception as e:
        print(e)
    finally:
        newParam = []
        newParam.append(m.hexdigest())
        saveList = orgParam + newParam
        cf.saveparam(saveList, SITENAME,DATANAME,SERVICENAME)
        print("saved")


In [64]:
# import pygame
# import math
# import numpy as np

numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
resultDf = pd.DataFrame()
recordIdx = 0
for i in range(0,dfCombiLen): 
    searchCstrNm = dfCombi.loc[i].cstrnm
    pageNo = 1
    print(searchCstrNm,"<>",i)

    if searchCstrNm != searchCstrNm:
        print("Nan Case")
        continue
    if len(searchCstrNm) <= 1:
        print("건설사명 이상: ",searchCstrNm)
        continue
#     print(searchCstrNm,"<>",i)
    g_totalCount = 0
    while True:
        apiNo = i%APIKEYLEN

        BASEPARAM={"serviceKey":APIKEY[apiNo],"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": startDate,"searchRpcdIsEdDt": endDate, "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
        print("BASEPARAM ",BASEPARAM)
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY, "jsonabnormal")
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("No Data")
            break
        else:
#                 print(resultDf)
            resultDfMerged = resultDfMerged.append(resultDf) 
#             hashSaveParam(searchCstrNm,str(startYr)+"0101", str(endYr)+"1231", pageNo) # 해시값을 요청 param에 저장
            g_totalCount -= numOfRows
            print("Remain Count : ", g_totalCount)
            if(g_totalCount <= 0):    
                recordIdx = i
                break
            pageNo += 1
print("FINISHED")
# pygame.init()
# pygame.mixer.init()
# sounda= pygame.mixer.Sound("../test.wav")
# sounda.play()

현대건설(주) <> 0
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20220427', 'searchRpcdIsEdDt': '20220428', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
일신종합건설㈜ <> 1
BASEPARAM  {'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'searchCstrNm': '일신종합건설㈜', 'searchRpcdIsBgDt': '20220427', 'searchRpcdIsEdDt': '20220428', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}


KeyboardInterrupt: 

In [44]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

품질검사성적서 등록 목록 save compled


In [45]:
resultDfMerged.shape

(2488537, 27)

### 다음 회차 수행을 위해 현재 검색일자(종료점)를 저장한다. 
### 다음 회차 검색은 현재 저장한 검색일자가 시작점이 된다. 

In [ ]:
# 최종 파라미터 저장
# endDate = "20220425" # <- 최초 적재 완료후 주석 처리 예정
saveLastSearchDate(endDate)

In [ ]:
# 초기 데이터 적재분에 대한 기록 
# 시공사명 12000개, 2001년 ~ 2022년 4월 25일까지 적재한 기록을 CSV파일로 기록해 둠

# dfCombi = dfCombi.iloc[0:recordIdx]

# dfCombi['lstSrchDate'] = list(endDate for _ in range(dfCombi.shape[0]))  
# saveLastSearchDateByName(dfCombi)